In [ ]:
import json
from models import MonsterMetadata, StatData
from .graph import build_dungeon_graph

def get_mock_db():
    # 몬스터 Cost 공식: (HP * Atk * SpdFactor * AtkSpd) / 10
    # SpdFactor = Speed / 100
    data = [
        # ID 101: 고블린 (약함)
        # Cost: (100 * 10 * 1.0 * 1.0) / 10 = 100
        {'monsterId': 101, 'name': 'Goblin', 'hp': 100, 'attack': 10, 'speed': 100, 'attackSpeed': 1.0},
        
        # ID 102: 오크 (보통)
        # Cost: (300 * 20 * 1.0 * 1.0) / 10 = 600
        {'monsterId': 102, 'name': 'Orc', 'hp': 300, 'attack': 20, 'speed': 100, 'attackSpeed': 1.0},
        
        # ID 103: 다크나이트 (강함)
        # Cost: (500 * 30 * 1.5 * 1.2) / 10 = 2700
        {'monsterId': 103, 'name': 'DarkKnight', 'hp': 500, 'attack': 30, 'speed': 150, 'attackSpeed': 1.2},
    ]
    return {d['monsterId']: MonsterMetadata.from_dict(d) for d in data}

def get_mock_hero():
# 히로인 스탯 (지능 제외)
# 전투력 공식: ((힘20 + 기량20) * 10 * 공속1.0) + (HP 250 / 5)
# = (40 * 10) + 50 = 400 + 50 = 450점
    return [StatData(hp=250, strength=20, dexterity=20, attack_speed=1.0)]

def get_unreal_request():
    # 언리얼 입력 데이터 (지능 없음)
    return {
        "playerIds": [101],
        "rooms": [
            {"roomId": 1, "type": 1, "size": 3, "monsters": None},
            {"roomId": 2, "type": 0, "size": 2, "monsters": None}
        ]
    }

if __name__ == "__main__":
    print("[Prototype] 던전 밸런싱 테스트")
    
    initial_state = {
        "hero_stats": get_mock_hero(),
        "monster_db": get_mock_db(),
        "dungeon_data": get_unreal_request(),
        "difficulty_log": {}
    }
    
    app = build_dungeon_graph()
    result = app.invoke(initial_state)
    
    print("\n🔍 [Result] 최종 데이터:")
    for room in result['dungeon_data']['rooms']:
        if room['type'] == 1:
            print(f"\nRoom {room['roomId']} (Size {room['size']}):")
            monsters = room.get('monsters')
            if monsters:
                print(f" 몬스터 배치됨 ({len(monsters)}마리):")
                for m in monsters:
                    print(f"      - ID: {m['monsterId']}, Pos: ({m['posX']}, {m['posY']})")
            else:
                print("      - 배치 실패")
        else:
            print(f"\n💨 Room {room['roomId']}: Empty")

    # 검증:
    # 히로인(450점) * 1.2 = 540점 타겟
    # 오크(600점)가 가장 근접하므로 주로 선택되어야 함.

ImportError: attempted relative import with no known parent package